In [ ]:
import h5py
import glob
from datetime import datetime, timedelta
import numpy as np
import pylab as plt
import seaborn as sns
from astropy import constants, units
from matplotlib.dates import date2num
import matplotlib.dates as mdates
import pyvista
import vtk
import progressbar
import cdflib
import dateutil.parser
import scipy.stats
import pandas as pd

import os
gp_base='/glade/u/home/danieldas/scratch/geopack/'
os.environ['KPDATA_PATH'] = gp_base + '/kp'
os.environ['OMNIDATA_PATH'] = gp_base + '/omni'
os.environ['GEOPACK_PATH'] = gp_base + '/data'
import PyGeopack as gp

sys.path.append('/glade/u/home/danieldas/scratch/phd_stuff/daSilvaInvariants')
from dasilva_invariants.meshes import get_lfm_hdf4_data, get_tsyganenko_params


In [ ]:
sns.set_style('darkgrid')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
DPI = 500

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
goes13_files = glob.glob('/glade/u/home/danieldas/scratch//data/LFM-20131002_GOES13/dn_magn-l2-hires_g13_d201310*_v0_0_2.nc')
goes13_files.sort()
goes13_files = goes13_files[:6]
goes13_files

In [ ]:
delta_idx = 2 * 60  # 1 min at 0.5 sec resolution

goes13_times = []
goes13_b_gsm = []

for file in goes13_files:
    nc = h5py.File(file)
    goes13_times.extend([datetime(2000, 1, 1, 12) + timedelta(seconds=dt)
                         for dt in nc['time'][::delta_idx]])
    goes13_b_gsm.extend(nc['b_gsm'][::delta_idx, :])
    nc.close()
    
goes13_times = np.array(goes13_times)
goes13_b_gsm = np.array(goes13_b_gsm)

mask = np.linalg.norm(goes13_b_gsm,axis=1) < 5000
goes13_times = goes13_times[mask]
goes13_b_gsm = goes13_b_gsm[mask]

In [ ]:
delta_idx = 30 # once every 30 minutes

nc = h5py.File('/glade/u/home/danieldas/scratch//data/LFM-20131002_GOES13/dn_goes-l2-orb1m_g13_y2013_v0_0.nc')
goes13_ephem_times = np.array([datetime(2000, 1, 1, 12) + timedelta(seconds=dt) for dt in nc['time'][::delta_idx]])
goes13_ephem_gse = nc['gse_xyz'][::delta_idx]
goes13_ephem_sm = []

for i in range(goes13_ephem_times.shape[0]):
    time = goes13_ephem_times[i]
    date = int(time.strftime('%Y%m%d'))
    ut = int(time.strftime('%H')) + time.minute / 60

    tmp_sm = gp.Coords.ConvCoords(
       goes13_ephem_gse[i, 0],
       goes13_ephem_gse[i, 1],
       goes13_ephem_gse[i, 2],
       date, ut, CoordIn='GSE', CoordOut='SM'
    )
    
    tmp_sm /= constants.R_earth.to(units.km).value
    goes13_ephem_sm.append(tmp_sm)

nc.close()

goes13_ephem_sm = np.array(goes13_ephem_sm)
goes13_ephem_sm[:10]

In [ ]:
lfm_files = glob.glob('/glade/u/home/danieldas/scratch/data/LFM-20131002_RBSP/*mhd*.hdf')
lfm_files.sort()
lfm_files[:10]

In [ ]:
lfm_file_times = []

for file in lfm_files:
    toks = file.split('/')[-1].split('_')[-1].split('T')
    rest = toks[1].split('-')
    file_time = datetime(*map(int, toks[0].split('-')), int(rest[0]), int(rest[1]), int(rest[2][:2]))
    lfm_file_times.append(file_time)
    
lfm_file_times[:10]

In [ ]:
goes13_ephem_sm.shape

In [ ]:
lfm_b_gsm = []
lfm_b_gsm_times = []

bar = progressbar.ProgressBar()
for lfm_time, lfm_file in bar(list(zip(lfm_file_times[::8], lfm_files[::8]))):
    mesh = get_lfm_hdf4_data(lfm_file)
    date = int(lfm_time.strftime('%Y%m%d'))
    ut = int(lfm_time.strftime('%H')) + lfm_time.minute / 60
    pos = []
    
    for i in range(3):
        pos.append(
            np.interp(
                date2num(lfm_time),
                date2num(goes13_ephem_times),
                goes13_ephem_sm[:, i].flatten()
            )
        )

    points_search = pyvista.PolyData(np.array([pos]))

    interp = vtk.vtkPointInterpolator()  # uses linear interpolation by default
    interp.SetInputData(points_search)
    interp.SetSourceData(mesh)
    interp.Update()

    points_interp = pyvista.PolyData(interp.GetOutput())

    tmp_sm = (points_interp['B'] * units.G).to(units.nT).value
    tmp_sm = np.array(tmp_sm)[0]
    tmp_gsm = gp.Coords.ConvCoords(
        tmp_sm[0], tmp_sm[1], tmp_sm[2],
        date, ut, CoordIn='SM', CoordOut='GSM'
    )

    lfm_b_gsm.append(tmp_gsm)
    lfm_b_gsm_times.append(lfm_time)

lfm_b_gsm_times = np.array(lfm_b_gsm_times)
lfm_b_gsm = np.array(lfm_b_gsm)
lfm_b_gsm[:10]

In [ ]:
lfmrcm_files = glob.glob('/glade/u/home/danieldas/scratch//data/LFMRCM-20131002_RBSP/*mhd*.hdf')
lfmrcm_files.sort()
lfmrcm_files[:10]

In [ ]:
lfmrcm_b_gsm = []
lfmrcm_b_gsm_times = []

bar = progressbar.ProgressBar()
for lfm_time, lfm_file in bar(list(zip(lfm_file_times[::8], lfmrcm_files[::8]))):
    mesh = get_lfm_hdf4_data(lfm_file)
    date = int(lfm_time.strftime('%Y%m%d'))
    ut = int(lfm_time.strftime('%H')) + lfm_time.minute / 60
    pos = []
    
    for i in range(3):
        pos.append(
            np.interp(
                date2num(lfm_time),
                date2num(goes13_ephem_times),
                goes13_ephem_sm[:, i].flatten()
            )
        )

    points_search = pyvista.PolyData(np.array([pos]))

    interp = vtk.vtkPointInterpolator()  # uses linear interpolation by default
    interp.SetInputData(points_search)
    interp.SetSourceData(mesh)
    interp.Update()

    points_interp = pyvista.PolyData(interp.GetOutput())

    tmp_sm = (points_interp['B'] * units.G).to(units.nT).value
    tmp_sm = np.array(tmp_sm)[0]
    tmp_gsm = gp.Coords.ConvCoords(
        tmp_sm[0], tmp_sm[1], tmp_sm[2],
        date, ut, CoordIn='SM', CoordOut='GSM'
    )

    lfmrcm_b_gsm.append(tmp_gsm)
    lfmrcm_b_gsm_times.append(lfm_time)


lfmrcm_b_gsm_times = np.array(lfmrcm_b_gsm_times)
lfmrcm_b_gsm = np.array(lfmrcm_b_gsm)
lfmrcm_b_gsm[:10]

In [ ]:
cdf = cdflib.CDF('/glade/u/home/danieldas/scratch//data/LFM-20131002_GOES13/omni_hro_1min_20131001_v01.cdf')
omni_epoch = np.array(
    [dateutil.parser.parse(t) for t in cdflib.epochs.CDFepoch().encode(cdf.varget('Epoch'))]
)

fill_value = 9999.99

omni_Bx = cdf.varget('BX_GSE')
omni_Bx[omni_Bx==fill_value] = np.nan

omni_By = cdf.varget('BY_GSM')
omni_By[omni_By==fill_value] = np.nan

omni_Bz = cdf.varget('BZ_GSM')
omni_Bz[omni_Bz==fill_value] = np.nan

symh = cdf.varget('SYM_H')

fill_value = 999.99
omni_n = cdf.varget('proton_density')
omni_n[omni_n==fill_value] = np.nan

fill_value = 99999.99
omni_speed = cdf.varget('flow_speed')
omni_speed[omni_speed==fill_value] = np.nan

mask = np.isfinite(omni_Bx) & np.isfinite(omni_By) & np.isfinite(omni_Bz) & np.isfinite(omni_n) & np.isfinite(omni_speed)

omni_epoch = omni_epoch[mask]
omni_Bx = omni_Bx[mask]
omni_By = omni_By[mask]
omni_Bz = omni_Bz[mask]
omni_n = omni_n[mask]
omni_speed = omni_speed[mask]
symh = symh[mask]

omni_dynpres = (constants.m_p * omni_n / units.cm**(3)) * (omni_speed * units.km/units.s)**2
omni_dynpres = omni_dynpres.to(units.nPa).value

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(16, 13))
axes[0].plot(omni_epoch, omni_Bx, label='Bx')
axes[0].plot(omni_epoch, omni_By, label='By')
axes[0].plot(omni_epoch, omni_Bz, label='Bz')
axes[0].plot(omni_epoch, np.sqrt(omni_Bx**2 + omni_By**2 + omni_Bz**2), label='|B|', color='k')
axes[0].legend(ncol=4)
axes[0].set_ylabel('$B$ (nT)')
axes[1].plot(omni_epoch, omni_n, color='c')
axes[1].set_ylabel('$n$ ($cm^{-3}$)')
axes[2].plot(omni_epoch, omni_speed, color='m')
axes[2].set_ylabel('$V_{sw}$ (km/s)')
axes[3].plot(omni_epoch, omni_dynpres, color='y')
axes[3].set_ylabel('$P_{dyn}$ (nPa)')
axes[4].plot(omni_epoch, symh, color='k')
axes[4].set_ylabel('$Dst$ (nT)')

for ax in axes:
    ax.set_xlim(lfm_file_times[0], lfm_file_times[-1])
    ax.axvline(datetime(2013, 10, 4, 0, 0), color='black', linestyle='dashed')
    ax.axvline(omni_epoch[np.argmin(symh)], color='black', linestyle='dashed')

for ax in axes:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b%d\n%H:%M'))

fig.suptitle('OMNI IMF Parameters during 2 October 2013 Storm', fontweight='bold')
fig.tight_layout()

In [ ]:
n=8
indeces = list(range(0, len(lfm_file_times), n))

positions = np.zeros((len(indeces), 3))
for i in range(3):
    positions[:, i] = np.interp(
        date2num(np.array(lfm_file_times)[indeces]).flatten(),
        date2num(goes13_ephem_times).flatten(),
        goes13_ephem_sm[:, i].flatten()
    )

In [ ]:

importlib.reload(meshes)
from dasilva_invariants.meshes import get_tsyganenko_params

In [ ]:
times_interp = [lfm_file_times[i] for i in indeces]
params_interp = meshes.get_tsyganenko_params(times_interp, path='/glade/u/home/danieldas/scratch/data/WGhour-latest.d.zip')

In [ ]:
params_anchor.keys()

In [ ]:
dates = [int(lfm_file_times[i].strftime('%Y%m%d')) for i in indeces]
uts = [int(lfm_file_times[i].strftime('%H')) + lfm_time.minute / 60 for i in indeces]

t96_times = np.array(lfm_file_times)[indeces]
t96_b_sm = gp.ModelField(positions[:, 0], positions[:, 1], positions[:, 2], Model='T96',
                         Date=dates, ut=uts,
                         CoordIn='SM', CoordOut='SM', **params_interp)
t04_times = t96_times.copy()
params = gp.Params.GetModelParams(dates, uts, 'TS05')
t04_b_sm =  gp.ModelField(positions[:, 0], positions[:, 1], positions[:, 2], Model='TS05',
                          Date=dates, ut=uts, CoordIn='SM',
                          CoordOut='SM', **params_interp)

t96_b_gsm = np.array(gp.Coords.SMtoGSM(*t96_b_sm, dates, uts)).T
t04_b_gsm = np.array(gp.Coords.SMtoGSM(*t04_b_sm, dates, uts)).T

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(20, 14))
fig.suptitle('Comparison of Magnetic Field Models During Storm along GOES13 Track', fontweight='bold')

Btotal_ax, Bx_ax, By_ax, Bz_ax, symh_ax = axes

if len(goes13_b_gsm.shape) == 3:
    goes13_b_gsm = goes13_b_gsm[:, :, 0]
if len(lfm_b_gsm.shape) == 3:
    lfm_b_gsm = lfm_b_gsm[:, :, 0]
if len(lfmrcm_b_gsm.shape) == 3:
    lfmrcm_b_gsm = lfmrcm_b_gsm[:, :, 0]
if len(t96_b_gsm.shape) == 3:
    t96_b_gsm = t96_b_gsm[:, :, 0]
if len(t04_b_gsm.shape) == 3:
    t04_b_gsm = t04_b_gsm[:, :, 0]
    
Btotal_ax.plot(goes13_times, np.linalg.norm(goes13_b_gsm, axis=1), color='k', label='GOES13')
Btotal_ax.plot(lfm_b_gsm_times, np.linalg.norm(lfm_b_gsm, axis=1), color='blue', label='LFM')
Btotal_ax.plot(lfm_b_gsm_times, np.linalg.norm(lfmrcm_b_gsm, axis=1), color='green', label='LFM+RCM')
Btotal_ax.plot(t96_times, np.linalg.norm(t96_b_gsm, axis=1), color='red', label='T96')
Btotal_ax.plot(t96_times, np.linalg.norm(t04_b_gsm, axis=1), color='purple', label='TS05')

Bx_ax.plot(goes13_times, goes13_b_gsm[:, 0], color='k', label='GOES13', linewidth=2)
By_ax.plot(goes13_times, goes13_b_gsm[:, 1], color='k', label='GOES13', linewidth=2)
Bz_ax.plot(goes13_times, goes13_b_gsm[:, 2], color='k', label='GOES13', linewidth=2)

Bx_ax.plot(lfm_b_gsm_times, lfm_b_gsm[:, 0], color='blue', label='LFM')
By_ax.plot(lfm_b_gsm_times, lfm_b_gsm[:, 1], color='blue', label='LFM')
Bz_ax.plot(lfm_b_gsm_times, lfm_b_gsm[:, 2], color='blue', label='LFM')

Bx_ax.plot(lfmrcm_b_gsm_times, lfmrcm_b_gsm[:, 0], color='green', label='LFM+RCM')
By_ax.plot(lfmrcm_b_gsm_times, lfmrcm_b_gsm[:, 1], color='green', label='LFM+RCM')
Bz_ax.plot(lfmrcm_b_gsm_times, lfmrcm_b_gsm[:, 2], color='green', label='LFM+RCM')

Bx_ax.plot(t96_times, t96_b_gsm[:, 0], color='red', label='T96')
By_ax.plot(t96_times, t96_b_gsm[:, 1], color='red', label='T96')
Bz_ax.plot(t96_times, t96_b_gsm[:, 2], color='red', label='T96')


Bx_ax.plot(t04_times, t04_b_gsm[:, 0], color='purple', label='TS05')
By_ax.plot(t04_times, t04_b_gsm[:, 1], color='purple', label='TS05')
Bz_ax.plot(t04_times, t04_b_gsm[:, 2], color='purple', label='TS05')


symh_ax.plot(omni_epoch, symh, color='orange')

Btotal_ax.set_ylabel('B (nT)')
Bx_ax.set_ylabel('$B_x$ (nT)')
By_ax.set_ylabel('$B_y$ (nT)')
Bz_ax.set_ylabel('$B_z$ (nT)')
symh_ax.set_ylabel('Dst (nT)')

myFmt = mdates.DateFormatter('%m/%d, %H:%M')

for ax in axes:
    ax.set_xlim(datetime(2013, 10, 2, 2, 0), lfm_b_gsm_times[-1])
    ax.xaxis.set_major_formatter(myFmt)
    ax.axvline(datetime(2013, 10, 4, 0, 0), color='black', linestyle='dashed')
    ax.axvline(omni_epoch[np.argmin(symh)], color='black', linestyle='dashed')
    
for ax in axes[:-1]:
    ax.legend(loc='upper right', ncol=1, bbox_to_anchor=(1.17, 1))
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(20, 8))
fig.suptitle('Comparison of Magnetic Field Models During Storm along GOES13 Track', fontweight='bold')

Bz_ax, symh_ax = axes

Bz_ax.plot(goes13_times, goes13_b_gsm[:, 2], 'k', label='GOES13', linewidth=4)
Bz_ax.plot(lfm_b_gsm_times, lfm_b_gsm[:, 2], color='blue', label='LFM', linewidth=2)
Bz_ax.plot(lfmrcm_b_gsm_times, lfmrcm_b_gsm[:, 2], color='green', label='LFM+RCM', linewidth=2)
Bz_ax.plot(t96_times, t96_b_gsm[:, 2], color='red', label='T96', linewidth=2)
Bz_ax.plot(t04_times, t04_b_gsm[:, 2], color='purple', label='TS05', linewidth=2)

symh_ax.plot(omni_epoch, symh, color='orange', linewidth=4)

Btotal_ax.set_ylabel('B (nT)')
Bx_ax.set_ylabel('$B_x$ (nT)')
By_ax.set_ylabel('$B_y$ (nT)')
Bz_ax.set_ylabel('$B_z$ (nT)')
symh_ax.set_ylabel('Dst (nT)')

myFmt = mdates.DateFormatter('%m/%d, %H:%M')

for ax in axes:
    ax.set_xlim(datetime(2013, 10, 2, 2, 0), datetime(2013, 10, 2, 7, 0))
    ax.xaxis.set_major_formatter(myFmt)
    ax.axvline(datetime(2013, 10, 4, 0, 0), color='black', linestyle='dashed')
    ax.axvline(omni_epoch[np.argmin(symh)], color='black', linestyle='dashed')

Bz_ax.legend(loc='upper right', ncol=6, bbox_to_anchor=(0.8, 1))
fig.tight_layout()